Lesson 15
# 課題: Web検索連動チャットボットを作成しよう

In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from typing import Annotated
from typing_extensions import TypedDict
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

# ===== Stateクラスの定義 =====
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ===== グラフの構築 =====
def build_graph(model_name):
    # ソースコードを記述
    # 検索ツールの定義
    tool = TavilySearchResults(max_results=2)
    tools = [tool]
    graph_builder = StateGraph(State)

    # 言語モデルの定義
    llm = ChatOpenAI(model_name=MODEL_NAME)
    llm_with_tools = llm.bind_tools(tools)

    # チャットボットノードの作成
    def chatbot(state: State):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

    # グラフにチャットボットノードを追加
    graph_builder.add_node("chatbot", chatbot)

    # ツールノードの作成
    tool_node = ToolNode(tools)

    # グラフにツールノードを追加
    graph_builder.add_node("tools", tool_node)

    # 条件付エッジの作成
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition, 
    )

    # ツールが呼び出されるたびに、チャットボットに戻って次のステップを決定
    # ツールからチャットボットへの戻りエッジを作成
    graph_builder.add_edge("tools", "chatbot")

    # 開始ノードの指定
    graph_builder.set_entry_point("chatbot")

    # 記憶を持つ実行可能なステートグラフの作成
    memory = MemorySaver()
    graph = graph_builder.compile(checkpointer=memory)
    return graph

# ===== グラフ実行関数 =====
def stream_graph_updates(graph, user_input: str):
    # 結果をストリーミングで得る
    # config引数でthread_idを指定してメモリ機能を有効化
    config = {"configurable": {"thread_id": "1"}}
    events = graph.stream({"messages": [("user", user_input)]}, config)
    for event in events:
        for value in event.values():
            if "messages" in value and value["messages"]:
                print(value["messages"][-1].content, flush=True)


# ===== メイン実行ロジック =====
# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini" 

# グラフの作成
graph = build_graph(MODEL_NAME)

# メインループ
while True:
    user_input = input("質問:")
    if user_input.strip() == "":
        print("ありがとうございました!")
        break
    print(user_input, flush=True)
    stream_graph_updates(graph, user_input)
    print("\n")

こんにちは！
こんにちは！どのようにお手伝いできますか？
1足す2は？
1 足す 2 は 3 です。何か他にも知りたいことがありますか？
ありがとうございました!
